# 分词统计报告

### 对爬取下来的数据分为个人服务和法人服务分别做分词工作

## 示例以个人服务来演示

### 查看总数据情况一共337447行

In [4]:
table_texts<-readLines("E:\\个人服务\\total.txt",skipNul=T)
length(table_texts)

[1] 337447

### 读取数据做数据清洗，去掉空格，去掉数字，和文件后缀

In [8]:
library(stringr)
library(dplyr)
library(jiebaR)
library(parallel)

table_texts<-readLines("E:\\个人服务\\total.txt",skipNul=T)
######################构造每一个函数
data_clean<-function(x){
  x<-gsub(" ","",x)
  x<-gsub(".txt","",x)
  x<-gsub("\\d","",x)
  x<-x
  return(x)
}

#################################集群操作#######################
#cl <- makeCluster(6) 
system.time({
     # 初始化四核心集群
    cl <- makeCluster(6)
   #调用parLapply并行计算函数
   results<-parLapply(cl,table_texts,data_clean)
#整合结果
final <- do.call('c',results)
# 关闭集群
stopCluster(cl) 
})



   user  system elapsed 
   0.32    0.13    7.46 

### 做了数据清洗后去掉小于两个字符的行数得到243730行

In [9]:
data<-final[nchar(final)>1]
length(data)

[1] 243730

### 开始分词(分词模型为mix混合模型，最后输出excel表格我会分别用最大概率模型（Maximum Probability），隐式马尔科夫模型（Hidden Markov Model），索引模型（QuerySegment），混合模型（MixSegment）这四种模型计算分词，分别对应生成各自的excel表格），当然这里也可以自己添加停用词库和使用词库，这里示例用混合模型和百度停用词库

In [15]:
engine<-worker(type="mix",stop_word = "C:\\Users\\lk\\Documents\\R\\Rpachong\\txt minging\\stopwords-master\\stopwords-master\\百度停用词表.txt")
data_new<-segment(data,engine)
length(data_new)

[1] 1017759

### 分词完成一共是1017759行

### 分词完成过后除去字符数字小于2的行数,得到938661行

In [16]:
data_final<-data_new[nchar(data_new)>1]
length(data_final)

[1] 938661

### 接下来我们进行词频统计（按出现次数最多到最低的排序），一共得到7729个词语,这里我们只显示前一百个词语，全部表格会在excel中

In [27]:
data_sort<-arrange(freq(data_final),desc(freq))
length(data_sort$char)
data_sort[1:100,]

[1] 7729

,char,freq
,<chr>,<dbl>
1,材料,38334
2,办理,19638
3,纸质,14231
4,名称,14166
5,申请人,11590
6,受理,11534
7,提交,9869
8,原件,9040
9,申请,8895


### 接下来进行词性标注，然后把词语，出现次数和词性标注整合到一个表里，导出成excel

In [29]:
cc = worker()
data_hh<-vector_tag(data_sort$char, cc)
data_frame<-data.frame(词语=data_hh,词性=names(data_hh),出现次数=data_sort$freq)
######查看前一百行
data_frame[1:100,]

,词语,词性,出现次数
,<chr>,<chr>,<dbl>
1,材料,n,38334
2,办理,n,19638
3,纸质,n,14231
4,名称,n,14166
5,申请人,nr,11590
6,受理,v,11534
7,提交,v,9869
8,原件,n,9040
9,申请,v,8895


In [31]:
library(wordcloud2)  

In [32]:
wordcloud2(data_sort, size = 1,shape = 'star') 

HTML widgets cannot be represented in plain text (need html)

### 最后输出 excel表格，法人服务分词统计业务也按照此步骤完成